In [45]:
#Habilitar si usas google colab

#from google.colab import drive
#drive.mount('/content/drive')

In [49]:
import pandas as pd

# Lectura de la data

In [50]:
# Utilizando google colab
#data = pd.read_csv('/content/drive/MyDrive/IA/PracticaCalificada2-CC421/train.csv', chunksize=42000000)

# Desde local (jupyter notebook)
data = pd.read_csv('train.csv', chunksize=42000000)

In [ ]:
data.get_chunk()

In [51]:
df = data.get_chunk() 

In [52]:
print(df.head())

                             key  fare_amount          pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5  2009-06-15 17:26:21 UTC   
1    2010-01-05 16:52:16.0000002         16.9  2010-01-05 16:52:16 UTC   
2   2011-08-18 00:35:00.00000049          5.7  2011-08-18 00:35:00 UTC   
3    2012-04-21 04:30:42.0000001          7.7  2012-04-21 04:30:42 UTC   
4  2010-03-09 07:51:00.000000135          5.3  2010-03-09 07:51:00 UTC   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.841610         40.712278   
1        -74.016048        40.711303         -73.979268         40.782004   
2        -73.982738        40.761270         -73.991242         40.750562   
3        -73.987130        40.733143         -73.991567         40.758092   
4        -73.968095        40.768008         -73.956655         40.783762   

   passenger_count  
0                1  
1                1  
2                2  
3       

# Preprocesamiento de la data

Se cuentan las filas con valores nulos, como estás representan una cantidad infíma
simplemente se eliminarán

In [53]:
print(df.isnull().sum())

key                    0
fare_amount            0
pickup_datetime        0
pickup_longitude       0
pickup_latitude        0
dropoff_longitude    281
dropoff_latitude     281
passenger_count        0
dtype: int64


In [54]:
df.dropna(inplace = True)
print(df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


In [55]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')

In [56]:
df['pickup_hour'] =df['pickup_datetime'].dt.hour

In [57]:
print(df['pickup_hour'])

0           17
1           16
2            0
3            4
4            7
            ..
41999995    14
41999996    13
41999997    12
41999998    18
41999999     1
Name: pickup_hour, Length: 41999719, dtype: int64


In [58]:
print(df['passenger_count'].value_counts())

1      29051954
2       6195177
5       2978039
3       1843562
4        892903
6        889744
0        148256
208          46
9            13
7            12
8             9
129           1
51            1
49            1
34            1
Name: passenger_count, dtype: int64


In [59]:
df.loc[df.passenger_count > 7, 'passenger_count'] = 7

In [61]:
print(df['passenger_count'].value_counts())

1    29051954
2     6195177
5     2978039
3     1843562
4      892903
6      889744
0      148256
7          84
Name: passenger_count, dtype: int64


In [66]:
print(df.dtypes)
print(df.head())

key                          object
fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
pickup_hour                   int64
dtype: object
                             key  fare_amount     pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5 2009-06-15 17:26:21   
1    2010-01-05 16:52:16.0000002         16.9 2010-01-05 16:52:16   
2   2011-08-18 00:35:00.00000049          5.7 2011-08-18 00:35:00   
3    2012-04-21 04:30:42.0000001          7.7 2012-04-21 04:30:42   
4  2010-03-09 07:51:00.000000135          5.3 2010-03-09 07:51:00   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.841610         40.712278   
1        -74.016048        40.711303         -73.979268         40.782004   
2        -73.982

Se usarán las siguientes carácteristicas


In [64]:
X = df.drop(['key','pickup_datetime','fare_amount'],1)
y = df.fare_amount

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)